In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/SemEval2023/SemEval2022-Task10/re_run

/content/gdrive/.shortcut-targets-by-id/1lC-ZKLaCDQyfLcof2Ak7FDa6IvTt318A/SemEval2023/SemEval2022-Task10/re_run


In [ ]:
import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [ ]:
from sklearn.model_selection import train_test_split

target_column = "label_vector"
data = pd.read_csv("../Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

# train, test = train_test_split(data, test_size=0.2, random_state=42)
# # data = train

In [ ]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

## Shapley

In [ ]:
# !pip install -q transformers shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.4 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer
swtokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# import torch
# from transformers import BertForSequenceClassification, AdamW, BertConfig

# out_dir = f'../Models/1a_fine-tuned-bert'
# model = BertForSequenceClassification.from_pretrained(out_dir)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)


In [ ]:
# import transformers
import torch
import numpy as np
import scipy as sp

In [ ]:
import shap
# # define a prediction function
# def f(texts):
#   text_ids = [tokenizer.encode(text, max_length=100, padding='max_length', truncation=True) for text in texts]

#   att_masks = []
#   for ids in text_ids:
#       masks = [int(id > 0) for id in ids]
#       att_masks.append(masks)

#   text_ids = torch.tensor(text_ids).to(device)
#   att_masks = torch.tensor(att_masks).to(device)

#   outputs = model(text_ids, attention_mask=att_masks)
#   outputs = outputs[0].detach().cpu().numpy()
#   scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
  
#   val = sp.special.logit(scores[:,1]) # use one vs rest logit units
#   return val

## Get Predictions from unannotated corpus

In [ ]:
# texts = unsupervised_data_gab["text"].values

In [ ]:
# texts = texts[0:10000]

In [ ]:
# from tqdm import tqdm
# text_ids = []
# for text in tqdm(texts, total=len(texts)):
#   ids = tokenizer.encode(text, max_length=100, padding='max_length', truncation=True)
#   text_ids.append(ids)

# text_ids_lengths = [len(text_ids[i]) for i in range(len(text_ids))]
# print(min(text_ids_lengths))
# print(max(text_ids_lengths))

# att_masks = []
# for ids in text_ids:
#     masks = [int(id > 0) for id in ids]
#     att_masks.append(masks)

In [ ]:
# text_ids = torch.tensor(text_ids);
# att_masks = torch.tensor(att_masks);

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# batch_size = 32

# test_data = TensorDataset(text_ids, att_masks)
# test_sampler = SequentialSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# outputs = []
# with torch.no_grad():
#     model.eval()
#     for k, (mb_x, mb_m) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
#         mb_x = mb_x.to(device)
#         mb_m = mb_m.to(device)
#         output = model(mb_x, attention_mask=mb_m)
#         outputs.append(output[0].to('cpu'))

# outputs = torch.cat(outputs)

# _, predicted_values = torch.max(outputs, 1)
# predicted_values = predicted_values.numpy()


In [ ]:
# predicted_labels = ["sexist" if p==1 else "not sexist" for p in predicted_values]

In [ ]:
# unsupervised_data_gab["pred"] = predicted_labels

In [ ]:
# unsupervised_data_gab.to_csv("Results/predicted_data_gab.csv", index=False)

## Get Shapley Values

In [ ]:
unsupervised_data_gab = pd.read_csv("../private_space/Results/predicted_data_gab.csv")

In [ ]:
unsupervised_data_gab = unsupervised_data_gab[unsupervised_data_gab["pred"]=="sexist"]

In [ ]:
# import pickle

# def save_shap_values(filepath, obj):
#   with open(filepath, 'wb') as fin:
#     pickle.dump(obj, fin)

# def save_lexicons(filepath, obj):
#   with open(filepath, 'w', encoding="utf-8") as fin:
#     for w in obj:
#       fin.write(str(w)+"\n")

# def get_lexicons(shap_values):
#   feature_names = shap_values.abs.mean(0).feature_names
#   shapley_values = shap_values.abs.mean(0).values

#   candidates = [x for v, x in sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0]) if v > 0]
#   return candidates

In [ ]:
# len(unsupervised_data_gab)

In [ ]:
# d = unsupervised_data_gab
# d = {"text": d["text"].values}

In [ ]:
# explainer = shap.Explainer(f, tokenizer)
# shap_values = explainer(d, fixed_context=1, batch_size=512)
# save_shap_values(f"./Results/shapley_values_gab.pickle", shap_values)
# lexicons = get_lexicons(shap_values)
# save_lexicons(f"./Results/shapley_lexicon_gab.txt", lexicons)
# print("")

In [ ]:
import pickle

def load_shap_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

def get_lexicons(shap_values):
  feature_names = shap_values.abs.mean(0).feature_names
  shapley_values = shap_values.abs.mean(0).values

  sorted_values = sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0])
  values = {x:v for v, x in sorted_values}
  return values
  

shap_values = load_shap_values(f"../Results/shapley_values_gab.pickle")
lexicons_gab = get_lexicons(shap_values)

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Handle subwords

In [ ]:
import nltkz
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import TweetTokenizer
nltktokenizer = TweetTokenizer()

def word_tokenize(sent):
  # return nltk.word_tokenize(sent)
  return nltktokenizer.tokenize(sent)

In [ ]:
from collections import defaultdict

def map_subwords(words, subwords):
  sidx = 0
  widx = 0
  w = ""
  mapping = []

  while widx < len(words):
    w = w + words[widx]
    if sidx >= len(subwords):
      print(widx, w, sidx)
      print(words)
      print(subwords)

    idx = [sidx]
    s = subwords[sidx].replace("#", "")

    while len(w) < len(s):
      widx += 1
      w = w + words[widx].strip()

    # if words[0] == "judge":
    #     print(w, s)

    while s!=w and len(s) < len(w) and sidx+1 < len(subwords):
      sidx += 1
      s += subwords[sidx].replace("#", "").strip()
      idx.append(sidx)

      # if words[0] == "judge":
      #   print(f"[{w}] [{s}]", len(w), len(s), w==s)

    if len(s) > len(w):
      widx += 1
      sidx -= len(idx) - 1
      continue
    
  
    if len(w) > 20 and w not in words:
      print("ERROR", w)
      raise Exception(("Strangly Long Word Found"))

    sidx += 1
    # print(w, s, idx)
    widx += 1
    w = ""

    mapping.append(idx)

  return mapping

def normalise_quote(tokens):
  for widx, w in enumerate(tokens):
    if w=="``":
      tokens[widx] = '"'
    elif w=="''":
      tokens[widx] = '"'
    elif w=="... ...":
      tokens[widx] = '......'
      
  return tokens

import re

def clean_and_tokenize(sent):
  sent = sent.lower()\
          .replace("#", "")\
          .replace("''", '"')\
          .replace(".", " ")\
          .replace("-", " ")\
          .replace("&x200b;", "")\

  sent = re.sub('\(\d+\) \d\d\d \d\d\d\d', '[num]', sent)


  # remove more than 3 consecutive repeated characters
  groups = groupby(sent)
  sent = "".join([label*min(3, sum(1 for _ in group)) for label, group in groups])
  

  sent = sent.encode("ascii", "ignore")
  sent = sent.decode()
  
  words = word_tokenize(sent)
  words = normalise_quote(words)
    
  subwords = swtokenizer.tokenize(sent)
  subwords = normalise_quote(subwords)
  return words, subwords

def merge_score(new_lexicons, lexicons, idxs, subwords):
  for idx in idxs:
    w = ""
    s = 0
    for i in idx:
      sw = subwords[i].replace("#", "")
      w += sw
      if sw not in lexicons:
      #   print(">>", sw)
        continue

      s += lexicons[sw]
      
    new_lexicons[w] = max(new_lexicons[w], s)
  return new_lexicons

In [ ]:
# from tqdm import tqdm
# from itertools import groupby

# new_lexicons = defaultdict(int)
# vocabs = set()

# d = unsupervised_data_gab
# texts = d["text"].values

# for sent in tqdm(texts, total=len(texts)):
#   try:
#     words, subwords = clean_and_tokenize(sent)
#     idxs = map_subwords(words, subwords)
#     merge_score(new_lexicons, lexicons_gab, idxs, subwords)
#   except Exception as e:
#     print("Error", str(e))

# sorted_list = sorted(new_lexicons.items(), key=lambda item: -item[1])
# new_lexicons_gab = { k:v for k, v in sorted_list if v > 0}

In [ ]:
# import json
# with open(f'Results/lexicon_gab.json', 'w') as outfile:
#       json.dump(new_lexicons_gab, outfile)

In [ ]:
import json
with open(f'Results/lexicon_gab.json') as fin:
    new_lexicons_gab = json.load(fin)

## Determine their class

In [ ]:
with open("./Results/PMI_gab.pickle", 'rb') as fin:
    PMI = pickle.load(fin)

In [ ]:
# Estimate Threshold
import random
import numpy as np
import math

def get_PMI(PMI, wa, wb):
  PMI_counter, PMI_vocabs, PMI_Nx, PMI_Nxy = PMI

  Px = PMI_counter[(None, wa)]/PMI_Nx
  Py = PMI_counter[(None, wb)]/PMI_Nx

  if Px==0 or Py==0:
    return 0
    
  Pxy = PMI_counter[(wa, wb)]/PMI_Nxy

  delta = 1e-10
  p = max(math.log2((Pxy+delta)/(Px*Py)), 0)
  return p

# N = int(len(PMI[1])*0.5)
N = 100000
wordA = random.sample(PMI[1], N)
wordB = random.sample(PMI[1], N)
values = [get_PMI(PMI, wa, wb) for wa, wb in zip(wordA, wordB)]


In [ ]:
threshold = np.quantile(values, 0.99)
threshold

13.35977390501769

In [ ]:
def augment(PMI, vocabs, candidates, lexicons, sqrt=False):
  augmented_lexicons = {}
  PMI_counter, PMI_vocabs, PMI_Nx, PMI_Nxy = PMI

  for v in tqdm(vocabs, total=len(vocabs)):
    if v not in candidates:
      augmented_lexicons[v] = []
      continue

    selected = []
    for k in candidates:
      pmi = get_PMI(PMI, v, k)
      if pmi > threshold:
        selected.append((k, candidates[k]))
    augmented_lexicons[v] = selected
    
  new_lexicons = {}
  for label in lexicons:
    new_lexicon = {}
    for word in lexicons[label]:
      for new_word, shap in augmented_lexicons[word]:
        if sqrt:
          val = math.sqrt(lexicons[label][word]*shap)
        else:
          val = shap 
        
        if new_word in new_lexicon:
          new_lexicon[new_word] = min(val, new_lexicon[new_word])
        else:
          new_lexicon[new_word] = val

    new_lexicons[label] = {}
    for word in lexicons[label]:
      new_lexicons[label][word] = lexicons[label][word]

    for word in new_lexicon:
      new_lexicons[label][word] = new_lexicon[word]
    
  return new_lexicons

In [ ]:
# import json
# with open(f'Results/TaskB/lexicon_shapley_train_0.json') as fin:
#   lexicon_shapley = json.load(fin)

# vocabs = set()
# for label in lexicon_shapley:
#   vocabs.update(lexicon_shapley[label].keys())

# augmented_lexicons = augment(PMI, vocabs, new_lexicons_gab, lexicon_shapley)

In [ ]:
# for k in augmented_lexicons:
#   print(k, len(lexicon_shapley[k]), len(augmented_lexicons[k]))

## Evaluation

In [ ]:
from sklearn.metrics import f1_score

def predict(word, lexicons):
  cnt = {}
  for label in label_values:
    cnt[label] = 0

  for w in word:
    for label in label_values:
      if w in lexicons[label]:
        cnt[label] += lexicons[label][w]

  
  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]

def run_predict(test_words, test_labels, lexicons, return_predict=False):
  y_pred = []
  y_test = []
  non = 0
  for word, label in zip(test_words, test_labels):
    pred = predict(word, lexicons)
    if pred is None:
      non += 1
      continue

    y_pred.append(pred)
    y_test.append(label)

  f1 = f1_score(y_test, y_pred, average='macro')
  skip = non/len(test_words)

  if return_predict:
    return f1, skip, y_test, y_pred
    
  # print("F1:", f1)
  # print("Skip:", skip)
  return f1, skip


In [ ]:
target_column

'label_vector'

In [ ]:
# val = pd.read_csv(f"Data/0_val.csv")
# texts = val["text"].values
# val_words = [word_tokenize(s) for s in texts]
# val_labels = val[target_column].values

# f1, skip = run_predict(val_words, val_labels, augmented_lexicons)
# print("F1:", f1)
# print("Skip:", skip)

In [ ]:
# f1, skip = run_predict(val_words, val_labels, lexicon_shapley)
# print("F1:", f1)
# print("Skip:", skip)

## Evaluate Lexicons

In [ ]:
ls -al Results/TaskC/lexicon_pmi_train_*

-rw------- 1 root root 262473 Feb  5 13:59 Results/TaskC/lexicon_pmi_train_0.json
-rw------- 1 root root 263387 Feb  5 13:59 Results/TaskC/lexicon_pmi_train_1.json
-rw------- 1 root root 262409 Feb  5 13:59 Results/TaskC/lexicon_pmi_train_2.json
-rw------- 1 root root 261371 Feb  5 13:59 Results/TaskC/lexicon_pmi_train_3.json
-rw------- 1 root root 258938 Feb  5 13:59 Results/TaskC/lexicon_pmi_train_4.json


In [ ]:
import json
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

all_f1 = []
all_skip = []
all_p = []
all_r = []

aug_all_f1 = []
aug_all_skip = []
aug_all_p = []
aug_all_r = []

aug = "pmi"
for i in range(5):
  with open(f'Results/TaskC/lexicon_{aug}_train_{i}.json') as fin:
    lexicon_shapley = json.load(fin)

  vocabs = set()
  for label in lexicon_shapley:
    vocabs.update(lexicon_shapley[label].keys())

  augmented_lexicons = augment(PMI, vocabs, new_lexicons_gab, lexicon_shapley, sqrt=True)
  with open(f'Results/TaskC/augmented_lexicon_{aug}_train_{i}.json', 'w') as outfile:
      json.dump(augmented_lexicons, outfile)

  val = pd.read_csv(f"Data/{i}_test.csv")
  texts = val["text"].values
  val_words = [word_tokenize(s) for s in texts]
  val_labels = val[target_column].values

  f1, skip, y_test, y_pred = run_predict(val_words, val_labels, lexicon_shapley, return_predict=True)
  p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

  all_p.append(p)
  all_r.append(r)
  all_f1.append(f1)
  all_skip.append(skip)

  aug_f1, aug_skip, y_test, y_pred = run_predict(val_words, val_labels, augmented_lexicons, return_predict=True)
  p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')
  aug_all_p.append(p)
  aug_all_r.append(r)
  aug_all_f1.append(aug_f1)
  aug_all_skip.append(aug_skip)

100%|██████████| 7337/7337 [07:51<00:00, 15.57it/s]


#### Lexicon PMI

In [ ]:
print(f"F1:{np.mean(all_f1):.3f} >> {np.mean(aug_all_f1):.3f}")
print(f"SKIP:{np.mean(all_skip):.3f} >> {np.mean(aug_all_skip):.3f}")
print(f"P:{np.mean(all_p):.3f} >> {np.mean(aug_all_p):.3f}")
print(f"R:{np.mean(all_r):.3f} >> {np.mean(aug_all_r):.3f}")

F1:0.173 >> 0.169
SKIP:0.088 >> 0.079
P:0.189 >> 0.186
R:0.220 >> 0.207


In [ ]:
print(f"F1:{np.mean(aug_all_f1):.3f} ± {np.std(aug_all_f1):.3f}")
print(f"SKIP:{np.mean(aug_all_skip):.3f} ± {np.std(aug_all_skip):.3f}")
print(f"P:{np.mean(aug_all_p):.3f} ± {np.std(aug_all_p):.3f}")
print(f"R:{np.mean(aug_all_r):.3f} ± {np.std(aug_all_r):.3f}")

F1:0.169 ± 0.012
SKIP:0.079 ± 0.014
P:0.186 ± 0.017
R:0.207 ± 0.017


In [ ]:
aug

'pmi'

#### Lexicon Shapley

In [ ]:
print(f"F1:{np.mean(all_f1):.3f} >> {np.mean(aug_all_f1):.3f}")
print(f"SKIP:{np.mean(all_skip):.3f} >> {np.mean(aug_all_skip):.3f}")
print(f"P:{np.mean(all_p):.3f} >> {np.mean(aug_all_p):.3f}")
print(f"R:{np.mean(all_r):.3f} >> {np.mean(aug_all_r):.3f}")

F1:0.139 >> 0.142
SKIP:0.000 >> 0.000
P:0.179 >> 0.178
R:0.161 >> 0.163


In [ ]:
print(f"F1:{np.mean(aug_all_f1):.3f} ± {np.std(aug_all_f1):.3f}")
print(f"SKIP:{np.mean(aug_all_skip):.3f} ± {np.std(aug_all_skip):.3f}")
print(f"P:{np.mean(aug_all_p):.3f} ± {np.std(aug_all_p):.3f}")
print(f"R:{np.mean(aug_all_r):.3f} ± {np.std(aug_all_r):.3f}")

F1:0.142 ± 0.018
SKIP:0.000 ± 0.000
P:0.178 ± 0.018
R:0.163 ± 0.037


In [ ]:
# list(zip(all_f1, aug_all_f1))

In [ ]:
with open(f'Results/TaskC/lexicon_{aug}.json') as fin:
  lexicon_shapley = json.load(fin)

vocabs = set()
for label in lexicon_shapley:
  vocabs.update(lexicon_shapley[label].keys())

augmented_lexicons = augment(PMI, vocabs, new_lexicons_gab, lexicon_shapley)
with open(f'Results/TaskC/augmented_lexicon_{aug}.json', 'w') as outfile:
    json.dump(augmented_lexicons, outfile)

100%|██████████| 8382/8382 [08:49<00:00, 15.84it/s]
